In [ ]:
import tensorflow as tf

from azureml.core import Run
from azureml.core import Experiment, Workspace

In [18]:
ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='textgen')

run = Run(exp, 'textgen_1611659464_8c9caec1')

In [24]:
# create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/'):
        print(f.replace('outputs/model/', ""))
        output_path = os.path.join('./model/', f.replace('outputs/model/', ""))
        print('Downloading from {} to {} ...'.format(f, output_path))
        run.download_file(name=f, output_file_path=output_path)

one_step_model/saved_model.pb
one_step_model/variables/variables.data-00000-of-00001
one_step_model/variables/variables.index
outputs/training_checkpoints/checkpoint
outputs/training_checkpoints/ckpt_1.data-00000-of-00001
outputs/training_checkpoints/ckpt_1.index


In [26]:
one_step_reloaded = tf.saved_model.load('model/one_step_model')

states1 = None
states2 = None
next_char = tf.constant(['sejm'])
result = [next_char]

for n in range(100):
  next_char, states1, states2 = one_step_reloaded.generate_one_step(next_char, states1=states1, states2=states2)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

sejmowi wydaje się , że nie było .
jak pan zainteresował się w stosunku do takiego poglądu , że nie cho


In [37]:
model = run.register_model(model_name='textgen_model', model_path='outputs/model/')

In [38]:
model

Model(workspace=Workspace.create(name='textgen', subscription_id='09598021-a61a-4f74-bcdd-f6bf71f9396b', resource_group='textgen'), name=textgen_model, id=textgen_model:3, version=3, tags={}, properties={})

In [20]:
%%writefile gen.py
import json
import numpy as np
import os
import tensorflow as tf

from azureml.core.model import Model

def init():
    global model
    model_root = os.getenv('AZUREML_MODEL_DIR')
    model_folder = 'model/one_step_model'
    model = tf.saved_model.load(os.path.join(model_root, model_folder))
 
    
def run(input_):
    json_ = json.loads(input_)
    next_char = tf.constant([json_["data"]])
    n = json_['num']

    json_1 = { "data" : [] }
    for i in range(n):
        states = None
        next_char = tf.constant([json_["data"]])
        result = [ next_char ]
        for n in range(100):
            next_char, states = model.generate_one_step(next_char, states=states)
            result.append(next_char)
        json_1["data"].append(tf.strings.join(result)[0].numpy().decode("utf-8").replace('\r\n', " "))

    return json_1



Overwriting gen.py


In [ ]:
from azureml.core.runconfig import CondaDependencies

cd = CondaDependencies.create()
cd.add_conda_package('numpy')
cd.add_pip_package('tensorflow==2.4.1')
cd.add_pip_package("azureml-defaults")
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

In [36]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="gen.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=3.8,
                                               auth_enabled=True, 
                                               memory_gb=15,
                                               tags={'name': 'textgen', 'framework': 'TfKeras'},
                                               description='text-generation')

service = Model.deploy(workspace=ws, 
                           name='text-generation-c', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................

In [29]:
print(service.get_logs())

2021-01-26T22:46:29,910665491+00:00 - iot-server/run 
2021-01-26T22:46:29,915810227+00:00 - nginx/run 
2021-01-26T22:46:29,915810327+00:00 - gunicorn/run 
2021-01-26T22:46:29,912691366+00:00 - rsyslog/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-01-26T22:46:30,057568049+00:00 - iot-server/finish 1 0
2021-01-26T22:46:30,058938732+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (16)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 40
2021-01-26 22:46:30.845791: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /azureml-envs/azureml_849829b8f15789b45fa01a1c415d6f58/lib:/azureml-envs/azureml_849829b8f15789b45fa01a1c415d6f58/lib:
2021-01-26 22:46:30.846020: I tensorflow/stream_executor/cuda